**Set environment**

In [1]:
suppressWarnings(suppressMessages(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(FD_RES, "source", "CRISPRi_FlowFISH", "peak_casa_calls")
print(dir(fdiry))

 [1] "CAPRIN1_HCRFF_CASA_elem_quant.bed" "CAT_HCRFF_CASA_elem_quant.bed"    
 [3] "CD164_HCRFF_CASA_elem_quant.bed"   "ERP29_HCRFF_CASA_elem_quant.bed"  
 [5] "FADS1_HCRFF_CASA_elem_quant.bed"   "FADS2_HCRFF_CASA_elem_quant.bed"  
 [7] "FADS3_HCRFF_CASA_elem_quant.bed"   "FEN1_HCRFF_CASA_elem_quant.bed"   
 [9] "GATA1_HCRFF_CASA_elem_quant.bed"   "HBE1_HCRFF_CASA_elem_quant.bed"   
[11] "HBG1_HCRFF_CASA_elem_quant.bed"    "HBG2_HCRFF_CASA_elem_quant.bed"   
[13] "HBS1L_HCRFF_CASA_elem_quant.bed"   "HDAC6_HCRFF_CASA_elem_quant.bed"  
[15] "LMO2_HCRFF_CASA_elem_quant.bed"    "MEF2C_HCRFF_CASA_elem_quant.bed"  
[17] "MYB_HCRFF_CASA_elem_quant.bed"     "MYC_HCRFF_CASA_elem_quant.bed"    
[19] "NMU_HCRFF_CASA_elem_quant.bed"     "PVT1_HCRFF_CASA_elem_quant.bed"   


## Import data

In [3]:
fdiry = file.path(FD_RES, "source", "CRISPRi_FlowFISH", "peak_casa_calls")
fname  = "*HCRFF*bed"
fglob  = file.path(fdiry, fname)
fpaths = Sys.glob(fglob)

lst = lapply(fpaths, function(fpath){
    dat = read_tsv(fpath, col_names = FALSE, show_col_types = FALSE)
    return(dat)
})
dat = bind_rows(lst)

dat_peak_casa = dat
print(dim(dat))
head(dat)

[1] 113  25


X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
chr11,33944871,33945471,chr11:33944871-33945471:.,0.8037646,.,chr11:33944871-33945471:.,chr11,34051730,34051731,⋯,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA
chr11,34050971,34051471,chr11:34050971-34051471:.,0.9739873,.,chr11:34050971-34051471:.,chr11,34051730,34051731,⋯,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA
chr11,34051571,34054171,chr11:34051571-34054171:.,2.0642907,.,chr11:34051571-34054171:.,chr11,34051730,34051731,⋯,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA
chr11,34085871,34086871,chr11:34085871-34086871:.,1.0092724,.,chr11:34085871-34086871:.,chr11,34051730,34051731,⋯,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA
chr11,33944868,33945268,chr11:33944868-33945268:.,1.2146755,.,chr11:33944868-33945268:.,chr11,34438933,34438934,⋯,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA
chr11,34437468,34440568,chr11:34437468-34440568:.,3.2866681,.,chr11:34437468-34440568:.,chr11,34438933,34438934,⋯,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA


## Arrange table

In [4]:
dat = dat_peak_casa
dat %>% head %>% dplyr::select(X8:X25)

X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25
<chr>,<dbl>,<dbl>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
chr11,34051730,34051731,+,NA,NA,CAPRIN1,ENSG00000135387,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA
chr11,34051730,34051731,+,NA,NA,CAPRIN1,ENSG00000135387,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA
chr11,34051730,34051731,+,NA,NA,CAPRIN1,ENSG00000135387,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA
chr11,34051730,34051731,+,NA,NA,CAPRIN1,ENSG00000135387,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA
chr11,34438933,34438934,+,NA,NA,CAT,ENSG00000121691,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA
chr11,34438933,34438934,+,NA,NA,CAT,ENSG00000121691,NA,NA,TRUE,NA,NA,NA,NA,NA,TRUE,NA


In [5]:
dat = dat_peak_casa
dat = dat %>%
    dplyr::select(X1, X2, X3, X5, X14, X15)
colnames(dat) = c("Chrom", "Start", "End", "Score", "Gene_Symbol", "Gene_ENS")

dat = dat %>%
    dplyr::mutate(
        Name   = paste0(Chrom, ":", Start, "-", End),
        Strand = ".",
        Assay_Type  = "CRISPRi-HCRFF",
        Assay_Label = paste0("CRISPRi-HCRFF", ":", Gene_Symbol)
    ) %>%
    dplyr::select(
        Chrom, Start, End, Name, Score, Strand,
        Gene_Symbol, Gene_ENS,
        Assay_Type, Assay_Label
    ) 

dat_peak_casa_arrange = dat
print(dim(dat))
head(dat)

[1] 113  10


Chrom,Start,End,Name,Score,Strand,Gene_Symbol,Gene_ENS,Assay_Type,Assay_Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
chr11,33944871,33945471,chr11:33944871-33945471,0.8037646,.,CAPRIN1,ENSG00000135387,CRISPRi-HCRFF,CRISPRi-HCRFF:CAPRIN1
chr11,34050971,34051471,chr11:34050971-34051471,0.9739873,.,CAPRIN1,ENSG00000135387,CRISPRi-HCRFF,CRISPRi-HCRFF:CAPRIN1
chr11,34051571,34054171,chr11:34051571-34054171,2.0642907,.,CAPRIN1,ENSG00000135387,CRISPRi-HCRFF,CRISPRi-HCRFF:CAPRIN1
chr11,34085871,34086871,chr11:34085871-34086871,1.0092724,.,CAPRIN1,ENSG00000135387,CRISPRi-HCRFF,CRISPRi-HCRFF:CAPRIN1
chr11,33944868,33945268,chr11:33944868-33945268,1.2146755,.,CAT,ENSG00000121691,CRISPRi-HCRFF,CRISPRi-HCRFF:CAT
chr11,34437468,34440568,chr11:34437468-34440568,3.2866681,.,CAT,ENSG00000121691,CRISPRi-HCRFF,CRISPRi-HCRFF:CAT


## Data Dictionary

In [6]:
dat_cnames = tribble(
    ~Name,         ~Description,
    "Chrom",       "Chromosome",
    "Start",       "Start position",
    "End",         "End position",
    "Name",        "Name",
    "Score",       "Score",
    "Strand",      "[+-.]; Use '.' if no strand is assigned.",
    "Gene_Symbol", "Gene symbol;     Gene that is screened for CRISPRi-FlowFish",
    "Gene_Ensembl","Gene Ensembl ID; Gene that is screened for CRISPRi-FlowFish",
    "Assay_Type",  "Assay type",
    "Assay_Label", "Assay label"
)

dat_cnames

Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start position
End,End position
Name,Name
Score,Score
Strand,[+-.]; Use '.' if no strand is assigned.
Gene_Symbol,Gene symbol; Gene that is screened for CRISPRi-FlowFish
Gene_Ensembl,Gene Ensembl ID; Gene that is screened for CRISPRi-FlowFish
Assay_Type,Assay type


## Save the tables

In [7]:
fdiry = file.path(FD_RES, "results", "region", "annotation_crispri_hcrff")
fname = "description.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_cnames
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_crispri_hcrff/description.tsv"


In [8]:
fdiry = file.path(FD_RES, "results", "region", "annotation_crispri_hcrff")
fname = "crispri_hcrff.casa.bed.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_casa_arrange
dat = dat %>% dplyr::arrange(Chrom, Start, End)
write_tsv(dat, fpath, col_names = FALSE)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_crispri_hcrff/crispri_hcrff.casa.bed.gz"


In [9]:
fdiry = file.path(FD_RES, "results", "region", "annotation_crispri_hcrff")
fname = "crispri_hcrff.casa.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_casa_arrange
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_crispri_hcrff/crispri_hcrff.casa.tsv"
